# Load data

In [62]:
import pandas as pd
import numpy as np
pd.set_option('max.columns',200)
pd.set_option('max.rows',200)

In [2]:
# load data
city = pd.read_csv('../data/raw/worldcities.csv')
df = pd.read_csv('../data/raw/us-train.csv')

In [3]:
# select relevant cities
city_df = city[city['id'].astype('str').isin(['1840034016', '1840020491', '1840000494', 
                                '1840006060', '1840000455', '1840021543'])]
city_df = city_df.rename(columns={'id':'city_id', 'city':'city_name'})
city_df['city_id'] = city_df['city_id'].astype('str')

In [4]:
bnb_city_id = {'NYC':'1840034016', 'SF':'1840021543', 'DC':'1840006060', 'LA':'1840020491', 'Chicago':'1840000494', 
               'Boston':'1840000455'}

df['city_id'] = df['city'].map(bnb_city_id)

In [5]:
city_df= city_df[['city_id', 'lat', 'lng']]

In [6]:
df = pd.merge(df, city_df, how='left', on='city_id')

In [7]:
df.host_has_profile_pic = np.where(df.host_has_profile_pic=='t',1,0)
df.host_identity_verified = np.where(df.host_identity_verified=='t',1,0)
df.instant_bookable = np.where(df.instant_bookable=='t',1,0)
df.cleaning_fee = np.where(df.cleaning_fee=='True',1,0)

In [8]:
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,city_id,lat,lng
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,0,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,1,1,NaN,2012-03-26,0,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0,1840034016,40.6943,-73.9249
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,0,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,1,0,100%,2017-06-19,1,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0,1840034016,40.6943,-73.9249
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,0,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,1,1,100%,2016-10-25,1,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0,1840034016,40.6943,-73.9249
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,0,SF,This light-filled home-away-from-home is super...,NaN,1,1,NaN,2015-04-19,0,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0,1840021543,37.7562,-122.4430
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,0,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,1,1,100%,2015-03-01,1,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0,1840006060,38.9047,-77.0163


# Feature Engineering

### Distance of Airbnb to city center & relative distances

In [9]:
from geopy import distance

def dist_city_center(prop_lat, prop_lon, city_lat, city_lon):
    prop_coords = (prop_lat, prop_lon)
    city_coords = (city_lat, city_lon)
    dist = distance.great_circle(prop_coords, city_coords).kilometers
    return dist

In [10]:
df['cc_dist'] = df.apply(lambda x: dist_city_center(x.latitude, x.longitude, x.lat, x.lng), axis=1)

In [11]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
#     m = km * 0.621371
    
    return km

In [12]:
def num_close_airbnbs(y, df, dist_thresh):
    df['cur_lat'] = y.latitude
    df['cur_lon'] = y.longitude
    df['rel_dist'] = haversine_np(df['cur_lon'], df['cur_lat'], df['longitude'], df['latitude'])
    num_close = np.sum(np.where(df['rel_dist'] <= dist_thresh, 1, 0)) - 1
    return num_close

In [13]:
# takes ~20 mins to run
data = df.copy()
df['num_bnb_1km'] = df.apply(lambda y: num_close_airbnbs(y, data, 1), axis=1)

### Description Length

In [14]:
df['desc_len'] = df['description'].str.len()
df['name_len'] = df['name'].str.len()

### Ratio of Bedrooms to bathrooms

In [15]:
# impute values w mode
df['bathrooms'].fillna(df['bathrooms'].mode()[0], inplace=True)
df['bedrooms'].fillna(df['bedrooms'].mode()[0], inplace=True)
df['beds'].fillna(df['beds'].mode()[0], inplace=True)

In [16]:
df['bed_bath_ratio'] = round(df.bedrooms/df.bathrooms,2)

### How long has the host been hosting

In [17]:
# based on day data was collected
df['today'] = '2018-03-14'

In [18]:
df['host_since'] = pd.to_datetime(df['host_since'])
df['today'] = pd.to_datetime(df['today'])
df['years_of_hosting'] = round(((df['today'] - df['host_since']).dt.days)/365,1)

### Years since last review (benchmark date = 2018-03-14)

In [19]:
df['last_review'] = pd.to_datetime(df['last_review'])
df['years_since_lastreview'] = round(((df['today'] - df['last_review']).dt.days)/365,1)

In [20]:
# Dropping the column I made to get the dates
df.drop('today', axis=1, inplace=True)

In [21]:
# null values generated because there is no 'last_review'
df.years_since_lastreview.isnull().sum()

15827

### Change % to float for column 'host_response_rate'

In [22]:
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype('float')

In [23]:
df.shape

(74111, 39)

### Column 'host_has_profile_pic', 'host_since', 'host_identity_verified' all has 203 missing values

In [24]:
# Let's change all nan values to 'f' (no profile pic, identity not verified)
df.loc[df['host_has_profile_pic'].isnull(), 'host_has_profile_pic'] = 'f'
df.loc[df['host_identity_verified'].isnull(), 'host_identity_verified'] = 'f'

### Encode 'thumbnail_url' column to 1 or 0 (if you have thumbnail then 1 else 0)

In [25]:
# we won't using this as dummy variable
df.thumbnail_url.nunique()

65883

In [26]:
# If the value is not null, then we assign 1, else 0
df.loc[df['thumbnail_url'].notnull(), 'thumbnail_url'] = 1

In [27]:
df.loc[df['thumbnail_url'].isnull(), 'thumbnail_url'] = 0

In [28]:
# Now we have either 1 or 0
df.thumbnail_url.unique()


array([1, 0], dtype=object)

### Drop duplicate rows

In [29]:
duplicate_rows_new_df = df[df.duplicated()]
print('number of duplicate rows:', duplicate_rows_new_df.shape)

#drop the duplicate rows
df = df.drop_duplicates()

number of duplicate rows: (0, 39)


### Number of Beds to bedroom ratio

In [30]:
df['bed_bath_ratio'] = round(df.bedrooms/df.bathrooms,2)

In [31]:
# There are null values in 'bed_bath_ratio' because there are houses with no bathrooms (campers, private rooms..)
# Replace null values in 'bed_bath_ratio' with 0
df.loc[df['bed_bath_ratio'].isnull(), 'bed_bath_ratio'] = 0.00

In [32]:
df.loc[df['bed_bath_ratio']==np.inf, 'bed_bath_ratio'] = 0.00

### Keep necessary columns

In [33]:
filter_cols = ['log_price',
 'accommodates',
 'bathrooms',
 'host_has_profile_pic',
 'host_identity_verified',
 'host_response_rate',
 'instant_bookable',
 'number_of_reviews',
 'review_scores_rating',
 'thumbnail_url',
 'bedrooms',
 'beds',
 'cc_dist',
 'num_bnb_1km',
 'desc_len',
 'name_len',
 'bed_bath_ratio',
 'years_of_hosting',
 'years_since_lastreview',
 'amenities',
 'property_type',
 'room_type']

In [34]:
df = df[filter_cols]

## Amenities

In [35]:
def handle_entry(s):
    # remove curly brace
    s = s.strip('{').strip('}')
    s = s.lower()
    # split
    s_list = s.split(",")
    # remove extra quote
    for i, curr_s in enumerate(s_list):
        s_list[i] = s_list[i].strip('"')
    return s_list

In [36]:
df['amenities'] = df['amenities'].apply(lambda x: handle_entry(x))

In [37]:
amenity_dict = {'amenity_tv': ['cable tv','tv'], 
 'amenity_baby': ['baby bath','baby monitor', 'babysitter recommendations', 'changing table', 
                  "children’s books and toys", "children’s dinnerware", 'family/kid friendly',
                  'fireplace guards', 'crib', 'high chair', "pack ’n play/travel crib", 'table corner guards',
                  'outlet covers', 'stair gates'],
'amenity_bed': ['bed linens', "extra pillows and blankets",'firm mattress','firm matress','firm mattress'],
'amenity_accessibility': [' smooth pathway to front door','accessible-height bed','accessible-height toilet',
                          'disabled parking spot','fixed grab bars for shower & toilet','flat smooth pathway to front door',
                          'grab-rails for shower and toilet','ground floor access','handheld shower head',
                          'path to entrance lit at night','roll-in shower with chair','step-free access',
                          'well-lit path to entrance','wheelchair accessible','wide clearance to bed',
                          'wide clearance to shower & toilet','wide clearance to shower and toilet','wide doorway',
                          'wide entryway','wide hallway clearance'],
'amenity_aircondition': ['air conditioning'],
'amenity_bbqgrill': ['bbq grill'], 
'amenity_breakfast': ['breakfast'],
'amenity_buzzer_wireless_intercom':['buzzer/wireless intercom'],
'amenity_cleaning_bf_checkout': ['cleaning before checkout'],
'amenity_pets': ['cat(s)', 'dog(s)', 'other pet(s)', 'pets allowed', 'pets live on this property'],
'amenity_washer_dryer':['dryer','washer','washer / dryer'],
'amenity_ev_charger':['ev charger'],
'amenity_elevator':['elevator','elevator in building'],
'amenity_free_park':['free parking on premises','free parking on street'],
'amenity_game':['game console'],
'amenity_indoor_fireplace': ['indoor fireplace'],
'amenity_gym':['gym'],
'amenity_clothes':['iron', 'hangers'],
'amenity_essential':['essentials'],
'amenity_hot_water': ['hot water'],
'amenity_garden_backyard':['garden or backyard'],
'amenity_bathroom':['bath towel', 'hand or paper towel', 'toilet paper', 'hand soap', 'body soap', 
                    'bathtub', 'shampoo', 'bathtub with shower chair', 'hot tub', 'shampoo', 'hair dryer'],
'amenity_host_greet': ['host greets you'], 
'amenity_heating': ['heating'],
'amenity_kitchen': ['refrigerator', 'dishes and silverware', 'microwave', 'hot water kettle', 
                    'dishwasher', 'coffee maker', 'stove', 'oven', 'cooking basics'],
'amenity_lake': ['lake access'],
'amenity_work': ['laptop friendly workspace'],
'amenity_bedroom_lock': ['lock on bedroom door'],
'amenity_lockbox': ['lockbox', 'keypad'],
'amenity_longterm_stays': ['long term stays allowed'],
'amenity_luggage_dropoff': ['luggage dropoff allowed'],
'amenity_other': ['other'],
'amenity_paid_parking': ['paid parking off premises'],
'amenity_patio_balcony': ['patio or balcony'],
'amenity_wifi': ['pocket wifi', 'wireless internet', 'internet', 'ethernet connection', 'internet'],
'amenity_pool': ['pool'],
'amenity_privacy': ['private bathroom', 'private entrance', 'private living room', 'room-darkening shades'],
'amenity_safety': ['safety card', 'smart lock', 'smartlock', 'smoke detector', 
                   'fire extinguisher', 'first aid kit', 'window guards'],
'amenity_self_checkin': ['self check-in'],
'amenity_single_level_home': ['single level home'],
'amenity_smoking_allowed': ['smoking allowed'],
'amenity_suitable_for_events': ['suitable for events'],
'amenity_waterfront': ['waterfront'],
'amenity_window_guards': ['window guards'],
'amenity_self_checkin':['self check-in'],
'amenity_beach_essentials':['beach essentials'],
'amenity_beach_front':['beachfront'],
'amenity_doorman':['doorman','doorman entry'],
'amenity_24hour_checkin':['24-hour check-in']}

In [38]:
def ohe_amenities(amenity, x):
    if amenity in x:
        return 1
    else:
        return 0

In [39]:
for agroup in amenity_dict.keys():
    amenity_cols = amenity_dict[agroup]
    for amenity in amenity_cols:
        df[amenity] = df['amenities'].apply(lambda x: ohe_amenities(amenity, x))
    df[agroup] = df[amenity_cols].sum(axis=1)
    df = df.drop(columns=amenity_cols)
    df[agroup] = np.where(df[agroup] >= 1, 1, 0)

## One-hot-encoding

In [40]:
cate_vari = ['property_type','room_type']
for i in cate_vari:
    #train[i]=train[i].astype(str)
    df[i]=pd.Categorical(df[i])

In [41]:
#one hot encoding for categorical variables
from sklearn.preprocessing import OneHotEncoder

# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing column (label encoded values)
df_ohe = pd.DataFrame(enc.fit_transform(df[cate_vari]).toarray())
df_ohe.columns = enc.get_feature_names()

# merge with main df bridge_df on key values
df = df.join(df_ohe)

# drop other columns
df.drop(columns=cate_vari, inplace=True)
df = df.drop(columns=['amenities'])

## Train, Validation, Test Split

In [44]:
from sklearn.model_selection import train_test_split

In [51]:
y = df[['log_price']]
X = df.drop(columns=['log_price'])

In [53]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.40, random_state=0)

In [56]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.50, random_state=0)

In [58]:
X_train.shape

(44466, 104)

In [59]:
X_val.shape

(14822, 104)

In [60]:
X_test.shape

(14823, 104)

## Impute null values

In [65]:
# impute null values for each dataset
X_train['host_response_rate'] = X_train['host_response_rate'].fillna(np.mean(X_train['host_response_rate']))
X_train['review_scores_rating'] = X_train['review_scores_rating'].fillna(np.mean(X_train['review_scores_rating']))
X_train['years_of_hosting'] = X_train['years_of_hosting'].fillna(np.mean(X_train['years_of_hosting']))
X_train['years_since_lastreview'] = X_train['years_since_lastreview'].fillna(np.mean(X_train['years_since_lastreview']))

X_val['host_response_rate'] = X_val['host_response_rate'].fillna(np.mean(X_train['host_response_rate']))
X_val['review_scores_rating'] = X_val['review_scores_rating'].fillna(np.mean(X_train['review_scores_rating']))
X_val['years_of_hosting'] = X_val['years_of_hosting'].fillna(np.mean(X_train['years_of_hosting']))
X_val['years_since_lastreview'] = X_val['years_since_lastreview'].fillna(np.mean(X_train['years_since_lastreview']))


X_test['host_response_rate'] = X_test['host_response_rate'].fillna(np.mean(X_train['host_response_rate']))
X_test['review_scores_rating'] = X_test['review_scores_rating'].fillna(np.mean(X_train['review_scores_rating']))
X_test['years_of_hosting'] = X_test['years_of_hosting'].fillna(np.mean(X_train['years_of_hosting']))
X_test['years_since_lastreview'] = X_test['years_since_lastreview'].fillna(np.mean(X_train['years_since_lastreview']))

/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarnin

## Concat y values back

In [72]:
train = pd.concat([y_train, X_train], axis=1)
test = pd.concat([y_test, X_test], axis=1)
val = pd.concat([y_val, X_val], axis=1)

In [77]:
train.shape

(44466, 105)

In [78]:
test.shape

(14823, 105)

In [79]:
val.shape

(14822, 105)

## Export csvs

In [80]:
train.to_csv('../data/processed/us-train.csv', index=False)

In [81]:
test.to_csv('../data/processed/us-test.csv', index=False)

In [82]:
val.to_csv('../data/processed/us-val.csv', index=False)